<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Find the distance between smoker and not

In [22]:
import cv2
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import math
import os

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
@dataclass
class Person:
    x1: int
    y1: int
    x2: int
    y2: int
    is_smoking: bool
    confidence: float

In [25]:
def calculate_pixels_per_meter(image_width: int, original_width: int = 2048) -> float:
    """
    Calcola il fattore di scala basato sulle dimensioni originali dell'immagine
    Assumendo che nell'immagine originale 100 pixel = 1 metro
    """
    return (100 * image_width) / original_width

In [26]:
def extract_people_from_image(image_path: str) -> Tuple[List[Person], np.ndarray]:
    """
    Estrae le informazioni sulle persone dall'immagine e ritorna anche l'immagine
    """
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definizione range colori
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    lower_blue = np.array([100, 70, 50])
    upper_blue = np.array([130, 255, 255])

    # Creazione maschere
    red_mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    red_mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(red_mask1, red_mask2)
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    people = []

    # Processa contorni rossi (fumatori)
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in red_contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20:
            people.append(Person(
                x1=x, y1=y, x2=x+w, y2=y+h,
                is_smoking=True,
                confidence=0.98
            ))

    # Processa contorni blu (non fumatori)
    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in blue_contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20:
            people.append(Person(
                x1=x, y1=y, x2=x+w, y2=y+h,
                is_smoking=False,
                confidence=0.98
            ))

    return people, image

In [27]:
def calculate_center_point(person: Person) -> Tuple[float, float]:
    """Calcola il punto centrale di una persona"""
    center_x = (person.x1 + person.x2) / 2
    center_y = (person.y1 + person.y2) / 2
    return (center_x, center_y)


In [28]:
def calculate_distance(p1: Person, p2: Person, pixels_per_meter: float) -> float:
    """Calcola la distanza euclidea tra due persone"""
    c1 = calculate_center_point(p1)
    c2 = calculate_center_point(p2)

    pixel_distance = math.sqrt(
        (c2[0] - c1[0])**2 + (c2[1] - c1[1])**2
    )

    return pixel_distance / pixels_per_meter

In [29]:
def find_smoker_nonsmoker_distances(people: List[Person], pixels_per_meter: float) -> List[Tuple[Person, Person, float]]:
    """
    Trova tutte le distanze tra fumatori e non fumatori
    """
    smokers = [p for p in people if p.is_smoking]
    non_smokers = [p for p in people if not p.is_smoking]
    distances = []

    for smoker in smokers:
        for non_smoker in non_smokers:
            distance = calculate_distance(smoker, non_smoker, pixels_per_meter)
            distances.append((smoker, non_smoker, distance))

    return distances

In [30]:
def process_and_save_image(image_path: str, output_dir: str) -> None:
    """
    Processa un'immagine disegnando le distanze tra i centri
    delle bounding box di fumatori e non fumatori
    """
    os.makedirs(output_dir, exist_ok=True)

    # Estrai le persone e ottieni l'immagine
    people, image = extract_people_from_image(image_path)

    # Calcola il fattore di scala basato sulla larghezza dell'immagine
    pixels_per_meter = calculate_pixels_per_meter(image.shape[1])

    # Definizione colori
    RED = (0, 0, 255)     # BGR per rosso (fumatori)
    BLUE = (255, 0, 0)    # BGR per blu (non fumatori)
    YELLOW = (0, 255, 255) # BGR per giallo (centri)
    BROWN = (42, 42, 165)  # BGR per marrone (linee distanza)

    # Disegna le bounding box e i centri
    for person in people:
        color = RED if person.is_smoking else BLUE
        cv2.rectangle(image, (person.x1, person.y1), (person.x2, person.y2), color, 2)

        # Disegna il punto centrale
        center = calculate_center_point(person)
        cv2.circle(image, (int(center[0]), int(center[1])), 3, YELLOW, -1)

    # Calcola e disegna le distanze
    distances = find_smoker_nonsmoker_distances(people, pixels_per_meter)
    for smoker, non_smoker, distance in distances:
        s_center = calculate_center_point(smoker)
        ns_center = calculate_center_point(non_smoker)

        # Disegna la linea tra i centri
        cv2.line(image,
                (int(s_center[0]), int(s_center[1])),
                (int(ns_center[0]), int(ns_center[1])),
                BROWN, 2)

        # Aggiungi il testo della distanza
        mid_point = ((s_center[0] + ns_center[0])//2, (s_center[1] + ns_center[1])//2)
        cv2.putText(image,
                   f"{distance:.2f}m",
                   (int(mid_point[0]), int(mid_point[1])),
                   cv2.FONT_HERSHEY_SIMPLEX,
                   0.7,
                   BROWN,
                   2)

    output_path = os.path.join(output_dir, f"distances_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, image)
    print(f"Distanze calcolate usando {pixels_per_meter:.2f} pixels/metro")

In [31]:
def main():
    input_dir = '/content/drive/MyDrive/test_trained_person'
    output_dir = '/content/drive/MyDrive/distance_img_process'

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_dir, filename)
            process_and_save_image(image_path, output_dir)
            print(f"Processata immagine: {filename}")

if __name__ == "__main__":
    main()

Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained1.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained2.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained3.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained4.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained5.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained6.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained7.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained8.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained9.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained10.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained11.jpg
Distanze calcolate usando 100.00 pixels/metro
Processata immagine: trained12.jpg
Distanze calcolate usando 100.00 pixe